In [1]:
from pybaseball import *
import numpy as np
import pandas as pd
import pickle
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
# from '../src/cleanids' import *

from numpy import nan
import re

import matplotlib.pyplot as plt

In [4]:
dl_18 = pd.read_pickle('../pickles/filter_dl_18.pkl')
print(len(dl_18))
dl_18

332


,name,team,injury,days,list,startdate,enddate,moneyearned,last,first
0,erichanhold,NYM,Oblique,7,60,2018-09-25,10/1,20510,hanhold,eric
1,michaelfulmer,DET,Knee/Oblique,47,60,2018-09-21,10/110-day,145324,fulmer,michael
2,claybuchholz,TOR,Elbow/Oblique,47,60,2018-09-15,10/110-day,379055,buchholz,clay
3,luizgohara,ATL,Shoulder/Ankle,49,60,2018-09-14,10/110-day,146216,gohara,luiz
4,michaelkopech,CHW,Elbow Tommy John,24,60,2018-09-08,10/1,70320,kopech,michael
5,keurymella,CIN,Oblique,28,60,2018-09-04,10/1,82040,mella,keury
6,josephbiagini,TOR,Oblique,11,10,2018-09-04,9/14,32230,biagini,joseph
8,brockstewart,LAD,Oblique,30,60,2018-09-02,10/1,87900,stewart,brock
9,pablolopez,MIA,Shoulder,31,60,2018-09-01,10/1,90830,lopez,pablo
10,kendallgraveman,CHC,Elbow Tommy John,31,60,2018-09-01,10/1,393328,graveman,kendall


In [5]:
def clean_ids(file):
    ids = pd.read_pickle(file)
    ids['name_last'] = ids['name_last'].str.replace('.', '')
    ids['name_last'] = ids['name_last'].str.replace("'", '')
    ids['name_last'] = ids['name_last'].str.replace(' ', '')
    ids['name_last'] = ids['name_last'].str.strip()
    ids['name_first'] = ids['name_first'].str.replace('.', '')
    ids['name_first'] = ids['name_first'].str.replace("'", '')
    ids['name_first'] = ids['name_first'].str.replace(' ', '')
    ids['name_first'] = ids['name_first'].str.strip()

    ids['name'] = 0
    for idx, val in ids.iterrows():
        ids['name'][idx] = ids['name_first'][idx] + '' + ids['name_last'][idx]

    # ids = ids.drop(['name_last', 'name_first'], axis=1)
    return ids 


In [425]:
keys = clean_ids('../pickles/moreids.pkl')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
def clean_inj(file, year):
    inj = pd.read_csv(file)
    inj['Year'] = year
    inj['Status'] = inj.Status.str[9:]
    DL_df = inj.copy()
    DL_df['List'], DL_df['Status'] = DL_df['Status'].str.split('|', 1).str
    DL_df['Dates'], DL_df['Status'] = DL_df['Status'].str.split('|', 1).str
    DL_df['StartDate'], DL_df['EndDate'] = DL_df['Dates'].str.split('-', 1).str
    DL_df['Dates'] = DL_df['Dates'].str.replace('..-day', '')
    DL_df['MoneyEarned'] = DL_df['Money_Earned'].str[1:]
    DL_df['MoneyEarned'] = DL_df['MoneyEarned'].str.replace(',', '')
    del DL_df['Dates']
    del DL_df['Status']
    del DL_df['Money_Earned']
    DL_df['Player'] = DL_df['Player'].str.lower()
    DL_df['List'] = DL_df['List'].str.replace('-day', '', regex=True)
    dates = []
    df = DL_df
    df.columns = map(str.lower, df.columns)
    for idx, val in df.iterrows():
        val.startdate = str(val.year)+val.startdate
        dates.append(val.startdate)

    dates = [x.rstrip() for x in dates]
    dates = [x.replace(' ','/') for x in dates]
    df['startdate'] = dates
    df['startdate'] = pd.to_datetime(df['startdate'], infer_datetime_format=True)
    
    lasts = []
    firsts = []

    for row in df.iterrows():
        name = row[1][0]
        name = jr_replace(name)
        name = title_replace(name)
        index = row[0]
        first = name.split()[0]
        first = correction2(first).rstrip()
        firsts.append(first)
        last = name.split()[-1]
        lasts.append(last)
    
    df['last'] = lasts
    df['first'] = firsts
    df['player'] = df['player'].str.lower()
    df['player'] = df['player'].str.replace('.', '')
    df['player'] = df['player'].str.replace("'", '')
    df['player'] = df['player'].str.replace(' ', '')
    df.rename(columns={'player': 'name'}, inplace=True)
    
    return df

In [8]:

def jr_replace(x):
    match = re.sub(r'jr$',"",x)
    return match

def title_replace(x):
    match = re.sub(r'iii$',"",x)
    return match

def correction2(string):
    corstr = re.sub('\ +',' ',string)
    final = re.sub('\.','. ',corstr)
    return final

In [426]:
df_18 = clean_inj('../data/dl_list_2018.csv', 2018)

In [432]:
print(len(df_18))
df_18

333


,name,posititon,team,injury,days,year,list,startdate,enddate,moneyearned,last,first
0,erichanhold,P,NYM,Oblique,7,2018,60,2018-09-25,10/1,20510,hanhold,eric
1,michaelfulmer,SP,DET,Knee/Oblique,47,2018,60,2018-09-21,10/110-day,145324,fulmer,michael
2,claybuchholz,SP,TOR,Elbow/Oblique,47,2018,60,2018-09-15,10/110-day,379055,buchholz,clay
3,luizgohara,SP,ATL,Shoulder/Ankle,49,2018,60,2018-09-14,10/110-day,146216,gohara,luiz
4,michaelkopech,SP,CHW,Elbow Tommy John,24,2018,60,2018-09-08,10/1,70320,kopech,michael
5,keurymella,RP,CIN,Oblique,28,2018,60,2018-09-04,10/1,82040,mella,keury
6,josephbiagini,RP,TOR,Oblique,11,2018,10,2018-09-04,9/14,32230,biagini,joseph
7,josemujica,P,TB,Undisclosed,29,2018,60,2018-09-03,10/1,84970,mujica,jose
8,brockstewart,SP,LAD,Oblique,30,2018,60,2018-09-02,10/1,87900,stewart,brock
9,pablolopez,SP,MIA,Shoulder,31,2018,60,2018-09-01,10/1,90830,lopez,pablo


In [433]:
drop_lst = []
ind = None
merge = df_18.merge(keys, on='name', how='left')
print(len(merge))
to_add = merge[merge['mlb_played_last'].isnull()]
to_add


354


,name,posititon,team,injury,days,year,list,startdate,enddate,moneyearned,last,first,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last


In [429]:
tp = playerid_lookup('blackburn')
tp


Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,blackburn,NaN,-1,NaN,NaN,-1,NaN,NaN
1,blackburn,NaN,-1,NaN,NaN,-1,NaN,NaN
2,blackburn,NaN,-1,NaN,NaN,-1,NaN,NaN
3,blackburn,NaN,-1,NaN,NaN,-1,NaN,NaN
4,blackburn,NaN,-1,NaN,NaN,-1,NaN,NaN
5,blackburn,NaN,-1,NaN,NaN,-1,NaN,NaN
6,blackburn,NaN,-1,NaN,NaN,-1,NaN,NaN
7,blackburn,NaN,-1,NaN,NaN,-1,NaN,NaN
8,blackburn,NaN,-1,NaN,NaN,-1,NaN,NaN
9,blackburn,NaN,-1,NaN,NaN,-1,NaN,NaN


In [316]:
tp = tp.iloc[181]
tp

name_last               black
name_first                vic
key_mlbam              502195
key_retro            blacv001
key_bbref           blackvi01
key_fangraphs            9195
mlb_played_first         2013
mlb_played_last          2014
Name: 181, dtype: object

In [326]:
keys[keys['name'].str.contains('deduno')]

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last,name
308,deduno,sam,465679.0,dedus001,dedunsa01,5285.0,2010.0,2015.0,samdeduno


In [334]:
ip = df_15[df_15['name'].str.contains('garvin')]
ind = ip.index[0]
ip

,name,posititon,team,injury,days,year,list,startdate,enddate,moneyearned,last,first
27,graysongarvin,SP,TB,Shoulder,41,2015,60,2015-08-26,10/5,113693,garvin,grayson


In [335]:
# drop_lst = []
drop_lst.append(ind)

In [336]:
drop_lst

[27]

In [337]:
df_15 = df_15.drop(index=drop_lst)

In [87]:
def add_to_dict_df():
    add_frame = pd.DataFrame()
    for row in to_add.iterrows():
        name = row[1][0]
        first = row[1][11]
        index = row[0]
        lookup = keys[keys['name'].str.contains(to_add['last'].loc[index])]
        if len(lookup) == 1:
            ins_index = lookup.index[0]
            lookup.set_value(ins_index, 'name_first', first)
            add_frame = add_frame.append(lookup)
    return add_frame

In [128]:
def add_to_dict_manual():
    add_frame = pd.DataFrame()
    for row in to_add.iterrows():
        name = row[1][0]
        first = row[1][11]
        index = row[0]
        lookup = keys[keys['name'].str.contains(to_add['last'].loc[index])]
        print(lookup)
        ins_index = int(input())
        lookup.set_value(ins_index, 'name_first', first)
        add_frame = add_frame.append(lookup)
    return add_frame

In [363]:
# add_frame = add_to_dict_manual()

In [324]:
add_frame = add_to_dict_df()
# add_frame = add_frame.iloc[:1]
add_frame

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last,name
308,deduno,samuel,465679.0,dedus001,dedunsa01,5285.0,2010.0,2015.0,samdeduno


In [327]:
new_keys = pd.concat([keys, add_frame])
new_keys = new_keys.sort_values(by='name_last')
new_keys = new_keys.set_index(np.arange(len(new_keys)))
new_keys.to_pickle('../pickles/moreids.pkl')

In [328]:
keys = clean_ids('../pickles/moreids.pkl')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [329]:
keys.to_pickle('../pickles/moreids.pkl')

In [430]:

drop_list = []
for row in to_add.iterrows():
    index = row[0]
    name = row[1][0]
    lookup = df_18[df_18['name'].str.contains(to_add['name'].loc[index])]
    if len(lookup) == 1:
        ins_index = lookup.index[0]
        drop_list.append(ins_index)
#         lookup.set_value(ins_index, 'name_first', first)
#         add_frame = add_frame.append(lookup)
    
#     drop_list.append(index)
drop_list

[282, 296, 297, 315, 322, 338]

In [431]:
df_18 = df_18.drop(index=drop_list)

In [317]:
keys = keys.append(tp)

In [318]:
keys = keys.sort_values(by='name_last')
keys = keys.set_index(np.arange(len(keys)))
keys.to_pickle('../pickles/moreids.pkl')

In [64]:
add_frame = add_to_dict_df()
add_frame

""


In [436]:
dups = merge.duplicated(subset='name', keep=False)
print(len(merge[dups]))
merge[dups]

5


,name,posititon,team,injury,days,year,list,startdate,enddate,moneyearned,last,first,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
264,miguelgonzalez,SP,CHW,Undisclosed,162,2018,60,2018-04-23,10/1,4137156,gonzalez,miguel,gonzalez,miguel,456068.0,gonzm003,gonzami03,7024.0,2012.0,2018.0
265,miguelgonzalez,SP,CHW,Undisclosed,162,2018,60,2018-04-23,10/1,4137156,gonzalez,miguel,gonzalez,miguel,544838.0,gonzm004,gonzami04,4049.0,2013.0,2013.0
266,miguelgonzalez,SP,CHW,Undisclosed,162,2018,60,2018-04-23,10/1,4137156,gonzalez,miguel,gonzalez,miguel,646057.0,gonzm006,gonzami05,15675.0,2014.0,2014.0
276,joseramirez,RP,ATL,Shoulder,167,2018,60,2018-04-18,10/1,507346,ramirez,jose,ramirez,jose,608070.0,ramij003,ramirjo01,13510.0,2013.0,2019.0
277,joseramirez,RP,ATL,Shoulder,167,2018,60,2018-04-18,10/1,507346,ramirez,jose,ramirez,jose,542432.0,ramij004,ramirjo02,10171.0,2014.0,2018.0


In [437]:
merge = merge.drop(index=[265, 266, 276])
# merge = merge.drop(index=[147, 148, 244])
# merge = merge.drop(index=[45, 46, 178])
# merge = merge.drop(index=[17, 25, 26, 64, 65, 230])

In [127]:
# new_keys = pd.concat([keys, add_frame])
# new_keys = new_keys.sort_values(by='name_last')
# new_keys = new_keys.set_index(np.arange(len(new_keys)))
# keys.to_pickle('../pickles/moreids.pkl')

In [435]:
merge = merge[merge['mlb_played_last'] >= 2013] 
print(len(merge))

336


In [438]:
merge

,name,posititon,team,injury,days,year,list,startdate,enddate,moneyearned,last,first,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,erichanhold,P,NYM,Oblique,7,2018,60,2018-09-25,10/1,20510,hanhold,eric,hanhold,eric,621098.0,hanhe001,hanhoer01,18332.0,2018.0,2018.0
1,michaelfulmer,SP,DET,Knee/Oblique,47,2018,60,2018-09-21,10/110-day,145324,fulmer,michael,fulmer,michael,605242.0,fulmm001,fulmemi01,13218.0,2016.0,2018.0
2,claybuchholz,SP,TOR,Elbow/Oblique,47,2018,60,2018-09-15,10/110-day,379055,buchholz,clay,buchholz,clay,453329.0,buchc001,buchhcl01,3543.0,2007.0,2018.0
3,luizgohara,SP,ATL,Shoulder/Ankle,49,2018,60,2018-09-14,10/110-day,146216,gohara,luiz,gohara,luiz,627894.0,gohal001,goharlu01,15166.0,2017.0,2018.0
4,michaelkopech,SP,CHW,Elbow Tommy John,24,2018,60,2018-09-08,10/1,70320,kopech,michael,kopech,michael,656629.0,kopem001,kopecmi01,17282.0,2018.0,2018.0
5,keurymella,RP,CIN,Oblique,28,2018,60,2018-09-04,10/1,82040,mella,keury,mella,keury,622492.0,mellk001,mellake01,15964.0,2017.0,2018.0
6,josephbiagini,RP,TOR,Oblique,11,2018,10,2018-09-04,9/14,32230,biagini,joseph,biagini,joseph,607352.0,biagj001,biagijo01,13194.0,2016.0,2019.0
7,josemujica,P,TB,Undisclosed,29,2018,60,2018-09-03,10/1,84970,mujica,jose,mujica,jose,465629.0,mujie001,mujiced01,3970.0,2006.0,2017.0
8,brockstewart,SP,LAD,Oblique,30,2018,60,2018-09-02,10/1,87900,stewart,brock,stewart,brock,592779.0,stewb001,stewabr01,16727.0,2016.0,2019.0
9,pablolopez,SP,MIA,Shoulder,31,2018,60,2018-09-01,10/1,90830,lopez,pablo,lopez,pablo,641154.0,lopep002,lopezpa01,17085.0,2018.0,2019.0


In [439]:
print(len(merge))

333


In [440]:
merge.to_pickle('../pickles/2018_inj_agg_games.pkl')

In [116]:
merge[merge['moneyearned'].isnull()]

,name,posititon,team,injury,days,year,list,startdate,enddate,moneyearned,last,first,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
